In [6]:
## use conda  "base"   

import pandas as pd
import numpy as np
import folium
from folium.plugins import HeatMap
from matplotlib import cm, colors
from branca.colormap import LinearColormap
from datetime import datetime

# 讀取 baseline.csv
df = pd.read_csv("baseline.csv")

# 整理欄位名稱
df = df.rename(columns={
    "latitude": "lat",
    "longitude": "lon",
    "max_volume_db": "volume"
})

# 清理資料：去除 NaN / 無限值與 (0,0)
df = df.replace([np.inf, -np.inf], np.nan).dropna(subset=["lat", "lon", "volume"])
df = df[~((df["lat"] == 0) & (df["lon"] == 0))].copy()

# 地圖中心
center = [df["lat"].mean(), df["lon"].mean()]
m = folium.Map(location=center, zoom_start=19, control_scale=True, tiles="OpenStreetMap")

# 顏色映射（依音量著色）
vmin, vmax = float(df["volume"].min()), float(df["volume"].max())
norm = colors.Normalize(vmin=vmin, vmax=vmax)

# 點位圖層
points_layer = folium.FeatureGroup(name="音量點位", show=True)
for _, r in df.iterrows():
    color = colors.to_hex(cm.jet(norm(r["volume"])))
    folium.CircleMarker(
        location=[r["lat"], r["lon"]],
        radius=4,
        fill=True,
        color=color,
        fill_color=color,
        fill_opacity=0.9,
        tooltip=f"{r.get('timestamp','')}\nVol: {r['volume']} dB"
    ).add_to(points_layer)
points_layer.add_to(m)

# 熱度圖層（權重=音量）
HeatMap(
    df[["lat", "lon", "volume"]].values.tolist(),
    name="熱度圖 (權重=音量)",
    radius=18, blur=22, max_zoom=19, min_opacity=0.3, show=True
).add_to(m)

# 色帶圖例
colormap = LinearColormap([colors.to_hex(cm.jet(x)) for x in np.linspace(0, 1, 6)],
                          vmin=vmin, vmax=vmax).to_step(6)
colormap.caption = "Volume (dB)"
colormap.add_to(m)

# 圖層控制
folium.LayerControl(collapsed=False).add_to(m)

# 輸出 HTML
out_file = f"baseline_map_{datetime.now().strftime('%Y%m%d_%H%M%S')}.html"
m.save(out_file)
out_file



'baseline_map_20250901_110616.html'